## Capstone 1 - Part 2

In the first part of the capstone, we focused on Data Retrieval, Data Preprocessing, Feature Engineering and Exploratory Data Analysis using Python & its libraries. Now we are going to shift gears and gain insights into our HR Analytics dataset using SQL.

## TODO: Make use of SQL to do the following:

### Create a SQLITE3 DB using the CSV file (2 pts). Please refer this [link](https://www.linkedin.com/pulse/accessing-sqlite3-database-from-jupyter-notebook-using-varun-lobo/) and this [link](https://www.geeksforgeeks.org/how-to-import-csv-file-in-sqlite-database-using-python/) to know more.

### Calculate the Attrition Rate and summarize attrition (3 pts) by:
- Gender
- Department
- Age
- Average monthly income by job level
- Years at company

### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

### Effective Communication (2 pts)
- Please make use of markdown cells to communicate your thought process, why did you think of performing a step? what was the observation from the query? etc.
- The code should be commented so that it is readable for the reviewer.

### Grading and Important Instructions
- Each of the above steps are mandatory and should be completed in good faith
- Make sure before submitting that the code is in fully working condition
- It is fine to make use of ChatGPT, stackoverflow type resources, just provide the reference links from where you got it
- Debugging is an art, if you find yourself stuck with errors, take help of stackoverflow and ChatGPT to resolve the issue and if it's still unresolved, reach out to me for help.
- You need to score atleast 7/10 to pass the project, anything less than that will be marked required, needing resubmission.
- Feedback will be provided on 3 levels (Awesome, Suggestion, & Required). Required changes are mandatory to be made.
- For submission, please upload the project on github and share the link to the file with us through LMS.

#### Import Libraries

In [1]:
import sqlite3
import pandas as pd

In [2]:
# Create a dataframe
df=pd.read_csv(r"C:\Users\Harsha\Desktop\Elevate Me\HR-Analytics.csv")

#### Creating a database locally using SQLite3

In [3]:
conn=sqlite3.connect('HR.db')

#### Create an Employee Table to store the data-frame as a table in the HR-Database

In [4]:
df.to_sql(name="employee",con=conn,if_exists='replace',index=False)
conn.commit()

#### Print the first 5 rows to check if connection is established successfully

In [5]:
query="select * from employee LIMIT 5"
result=pd.read_sql_query(query,conn)

In [6]:
print(result)

   Age Attrition     BusinessTravel  DailyRate              Department  \
0   41       Yes      Travel_Rarely       1102                   Sales   
1   49        No  Travel_Frequently        279  Research & Development   
2   37       Yes      Travel_Rarely       1373  Research & Development   
3   33        No  Travel_Frequently       1392  Research & Development   
4   27        No      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
0                 1          2  Life Sciences              1               1   
1                 8          1  Life Sciences              1               2   
2                 2          2          Other              1               4   
3                 3          4  Life Sciences              1               5   
4                 2          1        Medical              1               7   

   ...  RelationshipSatisfaction StandardHours  StockOptionLevel  \
0  ...

#### Calculate Attrition Rate


In [7]:
query='''Select Attrition,
       concat(round(count(*)* 100.0/(Select count(*)*1.0 as Total FROM employee),1),'%') as Attrition_rate 
       FROM employee 
       GROUP BY Attrition'''


#### It counts the occurrences of each Attrition value and divides it by the total number of employees in the table (calculated by a subquery).
#### The result is multiplied by 100 to convert it to a percentage, and rounded to one decimal place.
#### The CONCAT function appends the "%" symbol to the calculated rate, and the final result is displayed as Attrition_rate.

In [8]:
result=pd.read_sql_query(query,conn)
result

,Attrition,Attrition_rate
0,No,83.9%
1,Yes,16.1%


#### This indicates that 83.9% of employees did not leave the company (Attrition = "No"), while 16.1% of employees left the company (Attrition = "Yes").

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857886?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NiwiaG1hYyI6ImUyNGU0YWRhNWQwMDkxNWE2NWNiY2EwNGFlNDNiMTBlYmYyNWY0YzAiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Gender

In [9]:
query='''SELECT 
        e.Attrition,
        e.Gender,
        COUNT(*) AS Count_gender,
        ROUND(COUNT(*) * 100.0 / t.Total_gender, 1) AS Attrition_by_gender
        FROM 
        employee e
        JOIN 
        (SELECT Gender, COUNT(*) AS Total_gender FROM employee GROUP BY Gender) t
        ON e.Gender = t.Gender
        GROUP BY 
        e.Attrition, e.Gender;
'''

#### The subquery (SELECT Gender, COUNT(*) AS Total_gender FROM employee GROUP BY Gender) calculates the total number of employees for each gender.
#### The main query joins the employee table with this subquery on the Gender column to get the total count per gender for each Attrition status.
#### The COUNT(*) * 100.0 / t.Total_gender computes the attrition rate as a percentage, and the result is grouped by both Attrition and Gender.

In [10]:
result=pd.read_sql_query(query,conn)
result

,Attrition,Gender,Count_gender,Attrition_by_gender
0,No,Female,501,85.2
1,No,Male,732,83.0
2,Yes,Female,87,14.8
3,Yes,Male,150,17.0


#### This table shows the distribution of employees by gender and attrition status. It indicates that 85.2% of female employees and 83.0% of male employees stayed with the company, while 14.8% of female employees and 17.0% of male employees left (attrition).

#### Sample output

![image](https://api-v4.skyprepapp.com/public_api/da/857882?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MiwiaG1hYyI6IjM1NDU5N2E4MjZmMTMxNWFhMTA4NzFiYTFkZjQ5YjBhNTYyOWYwMzgiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Dept

In [11]:
query='''SELECT 
        Department,Attrition,
        count(*) AS Department_attrition
        FROM
        employee
        GROUP BY
        Department, Attrition
'''

#### This query groups the data by both Department and Attrition to count the number of employees in each combination of department and attrition status. It calculates the number of employees in each department who either left (Attrition = 'Yes') or stayed (Attrition = 'No'), and labels this count as Department_attrition.

In [12]:
result=pd.read_sql_query(query,conn)
result

,Department,Attrition,Department_attrition
0,Human Resources,No,51
1,Human Resources,Yes,12
2,Research & Development,No,828
3,Research & Development,Yes,133
4,Sales,No,354
5,Sales,Yes,92


#### Human Resources has the highest attrition rate (12 out of 63 employees) i.e 19%, while Research & Development has the lowest attrition rate, with 133 employees leaving out of 961 i.e 13.8%

#### Sample Output

<img src="https://api-v4.skyprepapp.com/public_api/da/857884?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NCwiaG1hYyI6IjdmOTA4NzJjODZmYTgxOGUyMzkyYTBlZjhjYjljNjA4ZGM3NjkzMWYiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9" width="450" height="450">

#### Find Attrition by Age Groups

In [13]:
query= '''WITH CTE1 AS (SELECT 
    Attrition, 
    (CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 39 THEN '30-40'
        WHEN Age BETWEEN 40 AND 49 THEN '40-50'
        ELSE 'Over 50'
    END) AS Age_Group,
    COUNT(*) AS num
FROM 
    employee
GROUP BY 
    Attrition, Age_Group),

    
    CTE2 AS (select Age_Group, sum(num) AS Total FROM CTE1 GROUP BY Age_Group)

    
    Select c1.Attrition,c1.Age_Group,c1.num,round((c1.num*100.0)/c2.Total,2) AS percent_by_age from CTE1 AS c1
    INNER JOIN
    CTE2 AS c2
    ON c1.Age_Group=c2.Age_Group
'''

#### CTE1 groups the employees by Attrition and age ranges, counting the number of employees in each group, creating a new column Age_Group based on the Age field.
#### CTE2 calculates the total number of employees for each Age_Group across all attrition categories.
#### The final query joins CTE1 and CTE2, calculating the percentage of employees in each Age_Group for each Attrition status by dividing the count in CTE1 by the total in CTE2 and multiplying by 100.

In [14]:
result=pd.read_sql_query(query,conn)
result

,Attrition,Age_Group,num,percent_by_age
0,No,30-40,533,85.69
1,No,40-50,315,90.26
2,No,Over 50,150,86.71
3,No,Under 30,235,72.09
4,Yes,30-40,89,14.31
5,Yes,40-50,34,9.74
6,Yes,Over 50,23,13.29
7,Yes,Under 30,91,27.91


#### The data indicates that younger employees (Under 30) have a higher attrition rate (27.91%), while older employees (40-50 and Over 50) tend to stay longer, with lower attrition rates (9.74% and 13.29%, respectively).

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857885?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NSwiaG1hYyI6IjcwNGVlOWIwYzg1MmMwNmNhODg0NmYyZDFlNWE3OTU1MTFhMGVmYWYiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Monthly Income

In [15]:
query='''WITH CTE AS (SELECT 
        Department,JobLevel,
        round(Avg(MonthlyIncome),1) AS avg_income,
        round(Avg(CASE WHEN Attrition="Yes" THEN MonthlyIncome
                        ELSE Null
                   END),1) AS attrition_avg_income
        FROM
        employee
        GROUP BY
        Department, JobLevel
        Having attrition_avg_income is not null)

        select *, (attrition_avg_income-avg_income) AS difference from CTE
'''

#### The CTE calculates the average monthly income for each Department and JobLevel, and the average monthly income for employees with Attrition = "Yes"
#### The final SELECT retrieves the results from the CTE and calculates the difference between the average income for all employees and those with attrition.

In [16]:
result=pd.read_sql_query(query,conn)
result

,Department,JobLevel,avg_income,attrition_avg_income,difference
0,Human Resources,1,2733.2,2415.7,-317.5
1,Human Resources,3,9623.0,10216.0,593.0
2,Research & Development,1,2840.1,2687.4,-152.7
3,Research & Development,2,5291.2,5372.0,80.8
4,Research & Development,3,10170.5,9503.8,-666.7
5,Research & Development,4,15634.7,12169.0,-3465.7
6,Research & Development,5,19218.5,19550.0,331.5
7,Sales,1,2506.7,2373.4,-133.3
8,Sales,2,5746.1,5917.0,170.9
9,Sales,3,9282.3,9202.8,-79.5


#### The data shows that employees with attrition ("Yes") tend to have lower average monthly incomes in most departments and job levels, as evidenced by negative differences in many rows including high job level employees like like Research & Development (Job Level 4) and Sales (Job Level 4).
#### Some extreme high job levels, like Research & Development (Job Level 5) and Sales (Job Level 5), exhibit a positive difference, suggesting that most high-level positions are retained.

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857883?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MywiaG1hYyI6Ijg3NTU4ZDU1ZjRjN2U1YWI3ODQzYjM1NzFkNjBjMjEwNGY5NWI5ODUiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Years At Company

In [17]:
query= '''WITH CTE1 AS (SELECT  
    (CASE 
        WHEN YearsAtCompany < 2 THEN 'New Hires'
        WHEN YearsAtCompany BETWEEN 2 AND 5 THEN '2-5 years'
        WHEN YearsAtCompany BETWEEN 6 AND 10 THEN '6-10 years'
        WHEN YearsAtCompany BETWEEN 11 AND 20 THEN '11-20 years'
        ELSE 'Over 20 years'
    END) AS tenure_years,
    COUNT(*) AS num
FROM 
    employee
WHERE
    Attrition="Yes"
GROUP BY 
    Attrition, tenure_years)

    
    select *,round((num*100.0)/237.0,1) AS percent from CTE1
'''

#### The CTE categorizes employees with Attrition = "Yes" into tenure groups based on YearsAtCompany and counts the number of employees in each group (num).
#### The final SELECT calculates the percentage of employees in each tenure group who have attrition, dividing the count by 237 (Total Number of rows with Attrition="Yes") and rounding to one decimal place.

In [18]:
result=pd.read_sql_query(query,conn)
result

,tenure_years,num,percent
0,11-20 years,12,5.1
1,2-5 years,87,36.7
2,6-10 years,55,23.2
3,New Hires,75,31.6
4,Over 20 years,8,3.4


#### The highest attrition occurs among employees with 2-5 years of experience (36.7%) followed by New Hires with (31.6%), while employees with over 20 years of tenure have the lowest attrition (3.4%).

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857881?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MSwiaG1hYyI6IjFhYmY3NGI4MzQ1NzViMWZkNDJlMjcwYTUyOTQ0OWQwZjJjMjhmNWUiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

#### Why do more people over 50 years old leave the company than people who aged 40-50?

In [19]:
query= '''WITH CTE1 AS (SELECT 
    Attrition, 
    (CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 39 THEN '30-40'
        WHEN Age BETWEEN 40 AND 49 THEN '40-50'
        ELSE 'Over 50'
    END) AS Age_Group,
    COUNT(*) AS num,
    avg(YearsSinceLastPromotion) AS avg_YearsSinceLastPromotion
FROM 
    employee
GROUP BY 
    Attrition, Age_Group),

    
    CTE2 AS (select Age_Group, sum(num) AS Total FROM CTE1 GROUP BY Age_Group)

    
    Select c1.Attrition,c1.Age_Group,c1.num,round((c1.num*100.0)/c2.Total,2) AS percent_by_age, c1.avg_YearsSinceLastPromotion from CTE1 AS c1
    INNER JOIN
    CTE2 AS c2
    ON c1.Age_Group=c2.Age_Group
    WHERE c1.Attrition="Yes"
'''

#### This query calculates attrition statistics for employees grouped by age categories. It first uses CTE1 to categorize employees into age groups, count attrition cases, and calculate the average years since the last promotion. Then, CTE2 computes the total employees in each age group. Finally, the main query joins the CTEs to calculate the percentage of attrition within each age group and filters for employees who left the company (Attrition = "Yes").

In [20]:
result=pd.read_sql_query(query,conn)
result

,Attrition,Age_Group,num,percent_by_age,avg_YearsSinceLastPromotion
0,Yes,30-40,89,14.31,2.134831
1,Yes,40-50,34,9.74,2.911765
2,Yes,Over 50,23,13.29,3.739130
3,Yes,Under 30,91,27.91,0.945055


#### Employees over 50 years old leave the company more frequently than those aged 40-50 because they have the highest average years since their last promotion (3.74 years). This suggests potential dissatisfaction due to stagnation in career growth.

#### Why do people with higher pay still leave the company?

In [42]:
query='''select Attrition,EnvironmentSatisfaction ,count(*) AS Employee_cnt FROM employee
        where MonthlyIncome>9999 and Attrition="Yes"
        Group BY attrition, EnvironmentSatisfaction
'''

#### The query counts the number of employees earning more than $9,999 per month who left the company ("Attrition = Yes"), grouped by their level of EnvironmentSatisfaction.

In [43]:
result=pd.read_sql_query(query,conn)
result

,Attrition,EnvironmentSatisfaction,Employee_cnt
0,Yes,1,10
1,Yes,2,4
2,Yes,3,5
3,Yes,4,6


#### Employees with low EnvironmentSatisfaction (level 1) account for the highest attrition (10 employees), suggesting that dissatisfaction with the work environment is a significant factor in high-paying employees leaving the company.

In [51]:
query='''select Attrition,StockOptionLevel ,count(*) AS Employee_cnt FROM employee
        where MonthlyIncome>9999 and Attrition="Yes"
        Group BY attrition, StockOptionLevel
'''

#### The query counts the number of employees earning more than $9,999 per month who left the company ("Attrition = Yes"), grouped by their StockOptionLevel.

In [52]:
result=pd.read_sql_query(query,conn)
result

,Attrition,StockOptionLevel,Employee_cnt
0,Yes,0,14
1,Yes,1,9
2,Yes,2,1
3,Yes,3,1


#### Employees with minimal stock options (StockOptionLevel 0 & 1) have the highest attrition (23 employees), indicating that a lack of stock options may contribute significantly to high-paying employees leaving the company.

#### Which factors drive employees who work at company less than 5 years to leave?

In [56]:
df['OverTime'].unique()

array(['Yes', 'No'], dtype=object)

In [75]:
query= '''SELECT  
    (CASE 
        WHEN YearsAtCompany < 5 THEN 'Less than 5 years'
        WHEN YearsAtCompany BETWEEN 6 AND 10 THEN '6-10 years'
        WHEN YearsAtCompany BETWEEN 11 AND 20 THEN '11-20 years'
        ELSE 'Over 20 years'
    END) AS tenure_years,
    COUNT(*) AS num,
    count(CASE WHEN JobSatisfaction<=2 THEN 1 ELSE NULL END) AS Low_JobSatisfaction_Count,
    count(CASE WHEN JobInvolvement<=2 THEN 1 ELSE NULL END) AS Low_JobInvolvement_Count,
    count(CASE WHEN EnvironmentSatisfaction<=2 THEN 1 ELSE NULL END) AS Low_EnvironmentSatisfaction_Count
FROM 
    employee
WHERE
    Attrition="Yes"
GROUP BY 
    Attrition, tenure_years
'''

result=pd.read_sql_query(query,conn)
result

,tenure_years,num,Low_JobSatisfaction_Count,Low_JobInvolvement_Count,Low_EnvironmentSatisfaction_Count
0,11-20 years,12,5,8,6
1,6-10 years,55,22,17,25
2,Less than 5 years,141,66,65,73
3,Over 20 years,29,19,9,11


#### For employees with Less than 5 years tenure, the key factors driving attrition seem to be:

#### Low Job Satisfaction (66 employees)
#### Low Environment Satisfaction (73 employees)
#### Low Job Involvement (65 employees)
#### These factors have the highest counts, suggesting that dissatisfaction with the job environment, lack of engagement, and overall job dissatisfaction are strong drivers for high turnover in employees with shorter tenures at the company.